In [1]:
import cv2 as cv
import numpy as np
import os
from pathlib import Path

### PATHS

In [ ]:

TRAINING_PATH = '../antrenare/'

TEMPLATES_PATH = '../templates/'

SOLUTION_PATH = '../solutions_fake_test/'

TEST_PATH = '../evaluare/fake_test/'

FIRST_BOARD = cv.imread("../imagini_auxiliare/01.jpg")

### CONSTANTS

In [3]:


ALL_DIGITS = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,24,25,27,28,30,32,35,36,40,42,45,48,49,50,54,56,60,63,64,70,72,80,81,90]

DIGIT_DICT = {digit: 7 if digit > 0 and digit < 11 else 1 for digit in ALL_DIGITS}

BASE_MATRIX = np.ones((14,14),dtype=np.int16)
BASE_MATRIX = BASE_MATRIX *-1
BASE_MATRIX[6,6] = 1
BASE_MATRIX[6,7] = 2
BASE_MATRIX[7,6] = 3
BASE_MATRIX[7,7] = 4

SPECIAL_TILE_DICT = {'x3':[(0,0),(0,6),(0,7),(0,13),(6,0),(7,0),(6,13),(7,13),(13,0),(13,6),(13,7),(13,13)],
                     'x2':[(1,1),(2,2),(3,3),(4,4),(12,12),(11,11),(10,10),(9,9),(12,1),(11,2),(10,3),(9,4),(1,12),(2,11),(3,10),(4,9)],
                     '+':[(3,6),(4,7),(6,4),(7,3),(9,6),(10,7),(7,9),(6,10)],
                     '-':[(2,5),(2,8),(5,2),(5,11),(8,2),(8,11),(11,5),(11,8)],
                     'x':[(3,7),(4,6),(6,3),(4,7),(6,9),(7,10),(9,7),(10,6)],
                     '/':[(1,4),(1,9),(4,1),(4,12),(9,1),(9,12),(12,4),(12,9)]
                     }

In [4]:
def show_img(img,title):
    if img.shape[0] > 1000:
        img=cv.resize(img,(0,0),fx=0.20,fy=0.20)
    cv.imshow(title,img)
    cv.waitKey(0)
    cv.destroyAllWindows()

### Use hsv colors to diferentiate the board from the rest of the background and get a mask

In [5]:
def crop_image_hsv(img):
    low_yellow = (17, 0, 0)
    high_yellow = (116, 255, 255)

    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    
    mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)

    return mask_yellow_hsv

### Using the mask from the above function return only the board

In [6]:
def extrage_careu(original_img, cropped_mask_img):
    kernel = np.ones((3, 4), np.uint8)
    cropped_mask_img = cv.erode(cropped_mask_img, kernel)

    edges =  cv.Canny(cropped_mask_img,20,400)

    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 2800
    height = 2800
    
    image_copy = original_img.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),1)
    # show_img(image_copy,"A")

    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    result = cv.warpPerspective(original_img, M, (width, height))
    
    return result

### Get the difference between the 2 imgs

In [7]:
def get_boards_difference(resized_img1,resized_img2):
    crop_img1 = crop_image_hsv(resized_img1)
    careu_img1 = extrage_careu(original_img=resized_img1,cropped_mask_img=crop_img1)

    crop_img2 = crop_image_hsv(resized_img2)
    careu_img2 = extrage_careu(original_img=resized_img2,cropped_mask_img=crop_img2)
    
    result = cv.absdiff(careu_img2,careu_img1)

    return crop_img1,careu_img1,crop_img2,careu_img2,result
        

### Generate the lines that delimitate the squares in the cropped picture

In [8]:
def generate_lines():
    lines_horizontal=[]
    for i in range(370,2452,148):
        l=[]
        l.append((350,i))
        l.append((2440,i))
        lines_horizontal.append(l)
    
    lines_vertical=[]
    for i in range(357,2600,148):
        l=[]
        l.append((i,370))
        l.append((i,2440))
        lines_vertical.append(l)

    return lines_horizontal,lines_vertical

### find the coordinates of the new places piece

In [9]:
def find_new_piece_position(thresh,lines_horizontal,lines_vertical):
    max_i = 0
    max_j = 0
    max_medie = 0
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            patch = thresh[x_min:x_max, y_min:y_max].copy()
            Medie_patch=np.mean(patch)
            if Medie_patch > max_medie:
                max_i = i
                max_j = j
                max_medie = Medie_patch
    return max_i,max_j

### Extract the patch with the new piece and preprocess it for the template matching

In [10]:
def extract_and_preprocess_patch(i,j,lines_horizontal,lines_vertical,careu_img):
    patch = careu_img[lines_horizontal[i][0][1]:lines_horizontal[i+1][0][1],lines_vertical[j][0][0]:lines_vertical[j+1][0][0],:]

    patch = patch[5:-5,5:-5,:]

    patch_blur = cv.bilateralFilter(patch, 15, 75, 75)

    patch_gray = cv.cvtColor(patch_blur,cv.COLOR_BGR2GRAY)


    T, patch_threshOtsu = cv.threshold(patch_gray, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3,3))

    patch_open = cv.morphologyEx(patch_threshOtsu, cv.MORPH_OPEN, kernel)

    return patch_open

### Get the templates for each individual number

In [11]:
def write_digits_template(img,lines_horizontal,lines_vertical):
    index = 0
    for i in range(0,len(lines_horizontal)-2,2):
        for j in range(0,len(lines_vertical)-2,2):
            patch = extract_and_preprocess_patch(i,j,lines_horizontal,lines_vertical,img)
            medie_patch=np.mean(patch)
            gray_img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
            T, img_threshOtsu = cv.threshold(gray_img, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)
            medie_img = np.mean(img_threshOtsu)
            cv.imwrite(str(TEMPLATES_PATH+str(index)+".jpg"),patch)
            index += 1

### Check the neighbors of the found piece in order to template match with only the possible ones

In [12]:
def check_possible_operations(i,j,dir_i,dir_j,game_matrix,special_tile):
    possible_digits = []
    if i >= 0 and i < 14 and j >= 0 and j < 14 and  game_matrix[i,j] != -1:
        if i + dir_i >= 0 and i + dir_i < 14 and j + dir_j >= 0 and j + dir_j < 14 and game_matrix[i + dir_i,j + dir_j] != -1:
            
            if special_tile == '+' or special_tile == None:
                if game_matrix[i + dir_i,j + dir_j] + game_matrix[i,j] not in possible_digits:
                    possible_digits.append(game_matrix[i + dir_i,j + dir_j] + game_matrix[i,j])

            if special_tile == '-' or special_tile == None:
                if abs(game_matrix[i + dir_i,j + dir_j] - game_matrix[i,j]) not in possible_digits:
                    possible_digits.append(abs(game_matrix[i + dir_i,j + dir_j] - game_matrix[i,j]))

            if special_tile == 'x' or special_tile == None:
                if game_matrix[i + dir_i,j + dir_j] * game_matrix[i,j] not in possible_digits:
                    possible_digits.append(game_matrix[i + dir_i,j + dir_j] * game_matrix[i,j])

            if special_tile == '/' or special_tile == None:
                if game_matrix[i,j] != 0 and game_matrix[i + dir_i,j + dir_j] % game_matrix[i,j] == 0:
                    if game_matrix[i + dir_i,j + dir_j] // game_matrix[i,j] not in possible_digits:
                        possible_digits.append(game_matrix[i + dir_i,j + dir_j] // game_matrix[i,j])

                elif game_matrix[i + dir_i,j + dir_j] and game_matrix[i,j] % game_matrix[i + dir_i,j + dir_j] == 0:
                    if game_matrix[i,j] // game_matrix[i + dir_i,j + dir_j] not in possible_digits:
                        possible_digits.append(game_matrix[i,j] // game_matrix[i + dir_i,j + dir_j])
    
    return possible_digits

### Check if the possible piece values are in the available pieces

In [13]:
def get_possible_digits(i,j,game_matrix,available_digits):
    possible_digits = []

    special_tile = None

    if (i,j) in SPECIAL_TILE_DICT['+']:
        special_tile = '+'
    elif (i,j) in SPECIAL_TILE_DICT['-']:
        special_tile = '-'
    elif (i,j) in SPECIAL_TILE_DICT['x']:
        special_tile = 'x'
    elif (i,j) in SPECIAL_TILE_DICT['/']:
        special_tile = '/'

    possible_digits.append(check_possible_operations(i-1,j,-1,0,game_matrix,special_tile))
    possible_digits.append(check_possible_operations(i,j+1,0,1,game_matrix,special_tile))
    possible_digits.append(check_possible_operations(i+1,j,1,0,game_matrix,special_tile))
    possible_digits.append(check_possible_operations(i,j-1,0,-1,game_matrix,special_tile))


    possible_digits = sum(possible_digits,[]) #flatten the array

    for digit in possible_digits:
        if digit in available_digits.keys():
            if available_digits[digit] == 0:
                possible_digits.remove(digit)
        else:
            possible_digits.remove(digit)


    return possible_digits,available_digits

### Perform template matching between the images from the templates folder and the new found piece

In [14]:
def template_match(patch,possible_digits):
    templates = os.listdir(TEMPLATES_PATH)
    
    best_min_val = 99999999
    best_max_val = 0
    digit = -1

    top, bottom, left, right = 50, 50, 50, 50

    patch = cv.copyMakeBorder(patch,top, bottom, left, right,borderType=cv.BORDER_CONSTANT,value=[0, 0, 0])

    for template_name in templates:
        if int(template_name[:-4]) in possible_digits:

            template = cv.imread(TEMPLATES_PATH + template_name,cv.IMREAD_GRAYSCALE)

            _, template = cv.threshold(template, 50, 255, cv.THRESH_BINARY)

            possible_result = cv.matchTemplate(patch,template,cv.TM_SQDIFF)

            min_val, max_val, _, _ = cv.minMaxLoc(possible_result)


            if min_val < best_min_val:
                best_min_val = min_val
                digit = int(template_name[:-4])
                
    return digit       

### Code for testing only 2 consecutive images

In [25]:
# img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/antrenare/1_02.jpg"
img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/imagini_auxiliare/01.jpg"
img1 = cv.imread(img_path)
img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/antrenare/1_01.jpg"
img2 = cv.imread(img_path)
# img1_r=cv.resize(img1,(0,0),fx=0.20,fy=0.20)
# img2_r=cv.resize(img2,(0,0),fx=0.20,fy=0.20)
img1_r=img1.copy()
img2_r=img2.copy()
crop_img1,careu_img1,crop_img2,careu_img2,result = get_boards_difference(img1_r,img2_r)

lines_horizontal,lines_vertical = generate_lines()

# write_digits_template(careu_img1,lines_horizontal,lines_vertical)

# for line in  lines_vertical : 
#     cv.line(careu_img2, line[0], line[1], (0, 0, 255), 5)
# for line in  lines_horizontal : 
#     cv.line(careu_img2, line[0], line[1], (0, 255, 0), 5)

# v.imshow("preproc_careu1",preproc_careu1)
# show_img(careu_img2,"careu_img2")
# show_img(careu_img1,"preproc_careu2")
kernel = cv.getStructuringElement(cv.MORPH_RECT, (13,13))
result = cv.morphologyEx(result, cv.MORPH_OPEN, kernel)
cv.imwrite("result.jpg",result)
show_img(result,"result")
i,j = find_new_piece_position(result,lines_horizontal,lines_vertical)
# print(i+1,end="")
# print(chr(j+65))

# cv.imwrite("result.jpg",result)
show_img(careu_img2[lines_horizontal[i][0][1]:lines_horizontal[i+1][0][1],lines_vertical[j][0][0]:lines_vertical[j+1][0][0],:],"patch")

preproc_patch = extract_and_preprocess_patch(i,j,lines_horizontal,lines_vertical,careu_img2)
cv.imwrite("patch.jpg",careu_img2[lines_horizontal[i][0][1]:lines_horizontal[i+1][0][1],lines_vertical[j][0][0]:lines_vertical[j+1][0][0],:])
show_img(preproc_patch,"preprocess_patch")
cv.imwrite("preprocess_patch.jpg",preproc_patch)
game_matrix = BASE_MATRIX.copy()
available_digits = DIGIT_DICT.copy()
possible_digits, available_digits = get_possible_digits(i,j,game_matrix,available_digits)
digit = template_match(preproc_patch,possible_digits)
print(possible_digits)
# available_digits[digit] -= 1
game_matrix[i,j] = digit
available_digits
# # show_img(careu_img2,"careu_img2")
# # show_img(crop_img1,"crop_img1")
# cv.imwrite("careu.png",careu_img2)
# # cv.imshow("careu_img1",preproc_careu2)
# # cv.imshow("result",result)
# # lines_vertical
# lines_horizontal

[np.int16(6), np.int16(2), np.int16(8)]


{0: 1,
 1: 7,
 2: 7,
 3: 7,
 4: 7,
 5: 7,
 6: 7,
 7: 7,
 8: 7,
 9: 7,
 10: 7,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 1,
 20: 1,
 21: 1,
 24: 1,
 25: 1,
 27: 1,
 28: 1,
 30: 1,
 32: 1,
 35: 1,
 36: 1,
 40: 1,
 42: 1,
 45: 1,
 48: 1,
 49: 1,
 50: 1,
 54: 1,
 56: 1,
 60: 1,
 63: 1,
 64: 1,
 70: 1,
 72: 1,
 80: 1,
 81: 1,
 90: 1}

In [338]:
def create_solutions_folder():
    if not os.path.exists(SOLUTION_PATH):
        os.makedirs(SOLUTION_PATH)

In [339]:
def write_solution_files(img_name,i,j,digit):
    file_name = img_name[:-4]

    file = open(str(SOLUTION_PATH + file_name + ".txt"),"w")

    file.write(str(i+1)+chr(j+65)+" "+str(digit))

### Return the turns of each player and order in which they begin

In [340]:
def get_players_turs(run_path,turns_file):
    path = Path(run_path + turns_file)

    turns = []

    with path.open(mode="r", encoding="utf-8") as md_file:
        content = md_file.read()
        turns = [line for line in content.splitlines()]

    player = int(turns[0][6])

        
    for i in range(len(turns)):
        turns[i] = turns[i][-2:].strip()

    return turns, player

### Calculate the multipliers applyed to the individual score if more than one ecuation results in the new placed piece or if it is placed on a X2 or X3 tile

In [341]:
def get_score(digit,possible_digits,i,j):

    multiplier = 1

    if possible_digits.count(digit) > 1:
        multiplier *= possible_digits.count(digit)

    if (i,j) in SPECIAL_TILE_DICT['x2']:
        multiplier *= 2
    elif (i,j) in SPECIAL_TILE_DICT['x3']:
        multiplier *= 3

    return digit * multiplier

In [342]:
def write_score(file,turns,scores,player):
    filepath = Path(SOLUTION_PATH + file)
    
    with filepath.open("w", encoding ="utf-8") as f:
        for i in range(len(turns)):
            line = "Player" + str(2 if player % 2 == 0 else 1) + " " + str(turns[i]) + " " + str(scores[i])
            if i != len(turns) - 1:
                line += '\n'
            f.write(line)
            player += 1

## MAIN CELL

### Modify the path at the top of the notebook
### Run all the cells before running the following in order to make the project work

In [ ]:
previous_img = FIRST_BOARD

# run_path = TRAINING_PATH
run_path = TEST_PATH

files=sorted(os.listdir(run_path))
lines_horizontal, lines_vertical = generate_lines()

game_matrix = BASE_MATRIX.copy()
available_digits = DIGIT_DICT.copy()

create_solutions_folder()

game = 1

turns, player = get_players_turs(run_path,str(game)+"_turns.txt")

scores = []
score = 0

turn = 1
scores_index = 1

for file in files:
    if file[-3:]=='jpg':
        current_img = cv.imread(run_path + file)

        if str(game) != file[0]:
            game += 1

            previous_img = FIRST_BOARD
            game_matrix = BASE_MATRIX.copy()
            available_digits = DIGIT_DICT.copy()

            scores.append(score)

            write_score(str(game - 1) + "_scores.txt",turns,scores,player)

            turns, player = get_players_turs(run_path,str(game)+"_turns.txt")     

            scores = []
            scores_index = 1

            score = 0
            turn = 1

        crop_img1,careu_img1,crop_img2,careu_img2,result = get_boards_difference(previous_img,current_img)

        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (13,13))
        result = cv.morphologyEx(result, cv.MORPH_OPEN, kernel)

        i,j = find_new_piece_position(result,lines_horizontal,lines_vertical)

        preproc_patch = extract_and_preprocess_patch(i,j,lines_horizontal,lines_vertical,careu_img2)

        possible_digits, available_digits = get_possible_digits(i,j,game_matrix,available_digits)

        digit = template_match(preproc_patch,possible_digits)
        

        if  scores_index < len(turns) and str(turn) == turns[scores_index]:
            scores.append(score)
            scores_index += 1
            score = 0
            

        score += get_score(digit,possible_digits,i,j)

        turn += 1

        available_digits[digit] -= 1

        game_matrix[i,j] = digit

        print(file)
        write_solution_files(file,i,j,digit)
        previous_img = current_img
        
        
scores.append(score)
write_score(str(game) + "_scores.txt",turns,scores,player)
print("Job's done!")